In [2]:
import pickle
import matplotlib.pyplot as plt
from skimage.transform import resize
import cv2
import numpy as np
import os
from os import listdir
import os.path as op
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors


In [8]:
'''

SCRIPT TO REMOVE THE 20% IMAGES WITH THE SMALLEST NB OF NON-ZERO PIXELS

'''

with open('./data/GO_terms.pkl', 'rb') as handle:
    GO_terms = pickle.load(handle)

path = op.abspath('./data/allendata')
genes = [f for f in listdir(path)]

info={} #make a dictionary which will contain the image id, gene id, non-zero count and the image itself, and the GO terms as well
geneNb = 0
for i in range(geneNb, len(genes)):
    gene=genes[i]
    try:
        GO=GO_terms[gene]
    except:
        GO=[]
    gene_path = path+'/'+gene
    genes_images = [f for f in listdir(gene_path)]
    for gene_image in genes_images:
        file = gene_path+'/'+gene_image
        img = cv2.imread(file)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        nzCount = cv2.countNonZero(gray_img);
        info[gene_image]=[gene, nzCount, gray_img, GO]
    if geneNb%1000==0:
        print(geneNb)
    geneNb += 1


filename='data/data_info.pkl'
with open(filename, 'wb') as handle:
    pickle.dump(info, handle)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [3]:
filename='data/data_info.pkl'
with open(filename, 'rb') as handle:
    info=pickle.load(handle)
    
keys=info.keys()
GO_terms=[]
genes=[]
gene_image_file=[]
gene_image_nz=[]
images_list=[]
for key in keys:
    genes.append(info[key][0])
    gene_image_nz.append(info[key][1])
    gene_image_file.append(key)
    images_list.append(info[key][2])
    GO_terms.append(info[key][3])
    
#delete images/genes which are in the lowest 20% non-zero count
geneNb=len(info)
genes_array = np.array(genes)
image_files = np.array(gene_image_file)
image_nz = np.array(gene_image_nz)
k = int(geneNb/5)
idx = np.argpartition(image_nz, k)
id_smallest = idx[:k]

image_files_rem = image_files[idx[k:]]
image_nz_rem = image_nz[idx[k:]]
genes_rem = genes_array[idx[k:]]
GO_terms_rem=np.asarray(GO_terms)[idx[k:]]
images_list_rem=np.asarray(images_list)[idx[k:]]


subset_dict={'gene_id':genes_rem, 'GO_terms':GO_terms_rem, 'images':images_list_rem, 'file_id':image_files_rem}


filename='data/NZ_del20.pkl' #deleted 20% images with lower NZ count
with open(filename, 'wb') as handle:
    pickle.dump(subset_dict, handle)
